In [65]:
import numpy as np
import astropy.io.fits as fits
from astropy.table import Table
from astromodels import Powerlaw,  PointSource, SpectralComponent
import astropy.time as at
from datetime import datetime
from pyspi.utils.function_utils import find_response_version
from pyspi.utils.response.spi_response_data import ResponseDataRMF
from pyspi.utils.response.spi_response import ResponseRMFGenerator
from pyspi.utils.response.spi_drm import SPIDRM
from pyspi.utils.livedets import get_live_dets
import os
from MultinestClusterFit import powerlaw_binned_spectrum

In [66]:
# reference orbit
data_path = "crab_data/0374"

time_elapsed = 3000.

In [67]:
# Energy Bins
with fits.open(f"{data_path}/energy_boundaries.fits") as file:
    t = Table.read(file[1])
    energy_bins = np.append(t["E_MIN"], t["E_MAX"][-1])

In [68]:
# Pointings and Start Times
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    
    pointings = np.array(t["PTID_SPI"])
    
    time_start = np.array(t["TSTART"]) + 2451544.5
    time_start = [at.Time(f"{i}", format="jd").datetime for i in time_start]
    time_start = np.array([datetime.strftime(i,'%y%m%d %H%M%S') for i in time_start])

In [77]:
# Define Source and Spectrum

ra, dec = 10, -40
# K, piv, index = 6e-3, 40, -2
# K, piv, index = 6e-3, 40, 0
K, piv, index = 6e-3, 40, 2


    
pl = Powerlaw()
pl.piv = piv
pl.K = K
pl.index = index
component1 = SpectralComponent("pl", shape=pl)
source = PointSource("Test", ra=ra, dec=dec, components=[component1])

emod = np.geomspace(10, 3000, 200)
spec = source(emod)
spec_binned = powerlaw_binned_spectrum(emod, spec)
# spec_binned = (emod[1:]-emod[:-1])*(spec[:-1]+spec[1:])/2

In [78]:
# Define Background

background_counts = np.linspace(1000, 10, len(energy_bins)-1)
background_counts = np.repeat(background_counts[np.newaxis,:], len(pointings) * 85, axis=0)
background_counts = np.random.poisson(background_counts)

In [79]:
# Generate Source Counts

assert find_response_version(time_start[0]) == find_response_version(time_start[-1]), "Versions not constant"
version = find_response_version(time_start[0])
rsp_base = ResponseDataRMF.from_version(version)

source_counts = np.zeros((len(pointings)*85, len(energy_bins)-1), dtype=np.uint32)

for p_i, pointing in enumerate(pointings):
    time = time_start[p_i]
    dets = get_live_dets(time=time, event_types=["single"])
    
    rmfs = []
    for d in dets:
        rmfs.append(ResponseRMFGenerator.from_time(time, d, energy_bins, emod, rsp_base))
        
    sds = np.empty(0)
    for d in range(len(dets)):
        sd = SPIDRM(rmfs[d], ra, dec)
        sds = np.append(sds, sd.matrix.T)
    resp_mat = sds.reshape((len(dets), len(emod)-1, len(energy_bins)-1))
    
    count_rates = np.dot(spec_binned, resp_mat)
    
    for d_i, d in enumerate(dets):
        index = p_i * 85 + d
        source_counts[index,:] = np.random.poisson(count_rates[d_i,:] * time_elapsed)    

Using the irfs that are valid between 04/07/17 08:20:06 and 09/02/19 09:59:57 (YY/MM/DD HH:MM:SS)


In [80]:
total_counts = source_counts + background_counts

166

In [81]:
destination_folder = "simulated_data/test_ba_de_so_in"

if not os.path.exists(destination_folder):
    os.mkdir(destination_folder)

In [82]:
os.popen(f"cp {data_path}/energy_boundaries.fits {destination_folder}/energy_boundaries.fits")
os.popen(f"cp {data_path}/pointing.fits {destination_folder}/pointing.fits")

In [75]:
with fits.open(f"{data_path}/evts_det_spec.fits") as file:
    t = Table.read(file[1])
    
updated_counts = t.copy()

updated_counts["COUNTS"] = total_counts

hdu = fits.BinTableHDU(data=updated_counts, name="SPI.-OBS.-DSP")
hdu.writeto(f"{destination_folder}/evts_det_spec.fits")

In [76]:
with fits.open(f"{data_path}/dead_time.fits") as file:
    t = Table.read(file[1])
    
updated_times = t.copy()

updated_times["LIVETIME"] = np.full(len(updated_times), time_elapsed)

hdu = fits.BinTableHDU(data=updated_times, name="SPI.-OBS.-DTI")
hdu.writeto(f"{destination_folder}/dead_time.fits")